In [2]:
from tinydb import TinyDB, Query
from collections import defaultdict
from functools import reduce
import operator
import requests
import json

# Initialize the database


In [22]:
db = TinyDB('cases.json')

# Fetch and save cases for each year from 1990 to 2024
for year in range(1990, 2025):
    url = f"https://api.oyez.org/cases?per_page=0&filter=term:{year}"
    response = requests.get(url)
    if response.status_code == 200:
        cases = response.json()
        for case in cases:
            db.insert(case)
    else:
        print(f"Failed to fetch data for year {year}: {response.status_code}")


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x118dc8590>>
Traceback (most recent call last):
  File "/Users/roie.s.c/dev/pinecone-neo4j/.venv/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [ ]:
# Count the number of documents in the db
num_documents = len(db)
print(f"Number of documents in the database: {num_documents}")


Number of documents in the database: 2883


In [ ]:
related_cases_docs = db.search((Query().related_cases != None) & (Query().related_cases.any(Query().related_cases.exists())))

# Print the number of documents found
print(len(related_cases_docs), "documents with related_cases found")



0 documents with related_cases found


In [ ]:

# Get all documents
all_docs = db.all()

if all_docs:
    # Get the first document
    first_doc = all_docs[0]
    
    # Print the keys (fields) of the first document
    print("Fields in the document:")
    for key in first_doc.keys():
        print(f"- {key}: {type(first_doc[key]).__name__}")
    
    # Print a sample of the first document
    print("\nSample document:")
    print(json.dumps(first_doc, indent=2))
else:
    print("The database is empty.")

Fields in the document:
- ID: int
- name: str
- href: str
- view_count: int
- docket_number: str
- timeline: list
- question: str
- citation: dict
- term: str
- description: NoneType
- justia_url: str

Sample document:
{
  "ID": 53807,
  "name": "Oklahoma Tax Commission v. Citizen Band, Potawatomi Indian Tribe of Oklahoma",
  "href": "https://api.oyez.org/cases/1990/89-1322",
  "view_count": 0,
  "docket_number": "89-1322",
  "timeline": [
    {
      "event": "Argued",
      "dates": [
        663228000
      ],
      "href": "https://api.oyez.org/case_timeline/case_timeline/36496"
    },
    {
      "event": "Decided",
      "dates": [
        667548000
      ],
      "href": "https://api.oyez.org/case_timeline/case_timeline/36497"
    }
  ],
  "question": "",
  "citation": {
    "volume": "498",
    "page": "505",
    "year": "1991",
    "href": "https://api.oyez.org/case_citation/case_citation/17736"
  },
  "term": "1990",
  "description": null,
  "justia_url": "https://supreme.jus

In [3]:
import requests

# Create a separate database for storing fetched href data
expanded_cases_db = TinyDB('expanded_cases.json')

# Iterate over each document in the database
for doc in db:
    href = doc.get('href')
    if href:
        try:
            response = requests.get(href)
            if response.status_code == 200:
                href_data = response.text
                expanded_cases_db.insert({'href': href, 'data': href_data})
            else:
                print(f"Failed to fetch href {href}: {response.status_code}")
        except requests.RequestException as e:
            print(f"Error fetching href {href}: {e}")
    else:
        print("No href found in document")



NameError: name 'db' is not defined

In [4]:
edb = TinyDB('expanded_cases.json')

related_cases_docs = edb.search((Query().related_cases != None))

# Print the number of documents found
print(len(related_cases_docs), "documents with related_cases found")



0 documents with related_cases found


In [5]:

# Get all documents
all_docs = edb.all()

if all_docs:
    # Get the first document
    first_doc = all_docs[0]
    
    # Print the keys (fields) of the first document
    print("Fields in the document:")
    for key in first_doc.keys():
        print(f"- {key}: {type(first_doc[key]).__name__}")
    
    # Print a sample of the first document
    print("\nSample document:")
    print(json.dumps(first_doc, indent=2))
else:
    print("The database is empty.")

Fields in the document:
- href: str
- data: str

Sample document:
{
  "href": "https://api.oyez.org/cases/1990/89-1322",
  "data": "{\"ID\":53807,\"name\":\"Oklahoma Tax Commission v. Citizen Band, Potawatomi Indian Tribe of Oklahoma\",\"href\":\"https:\\/\\/api.oyez.org\\/cases\\/1990\\/89-1322\",\"view_count\":0,\"docket_number\":\"89-1322\",\"additional_docket_numbers\":null,\"manner_of_jurisdiction\":\"Writ of \\u003Ci\\u003Ecertiorari\\u003C\\/i\\u003E\",\"first_party\":\"Oklahoma Tax Commission\",\"second_party\":\"Citizen Band, Potawatomi Indian Tribe of Oklahoma\",\"timeline\":[{\"event\":\"Argued\",\"dates\":[663228000],\"href\":\"https:\\/\\/api.oyez.org\\/case_timeline\\/case_timeline\\/36496\"},{\"event\":\"Decided\",\"dates\":[667548000],\"href\":\"https:\\/\\/api.oyez.org\\/case_timeline\\/case_timeline\\/36497\"}],\"lower_court\":{\"ID\":14,\"name\":\"United States Court of Appeals for the Tenth Circuit\",\"href\":\"https:\\/\\/api.oyez.org\\/taxonomy\\/term\\/14\"},\"fa

In [6]:
final_db = TinyDB('final.json')

for doc in edb:
    data = doc.get('data')
    href = doc.get('href')
    if data:
        try:
            parsed_data = json.loads(data)
            parsed_data['href'] = href
            final_db.insert(parsed_data)
        except json.JSONDecodeError as e:
            print(f"Error parsing data for href {doc.get('href')}: {e}")
    else:
        print(f"No data found in document with href {doc.get('href')}")


In [7]:
db = TinyDB('final.json')

related_cases_docs = db.search((Query().related_cases != None))

# Print the number of documents found
print(len(related_cases_docs), "documents with related_cases found")



5 documents with related_cases found


In [8]:
len(db)

2883

In [9]:
all_docs = db.all()

if all_docs:
    # Get the first document
    first_doc = all_docs[0]
    
    # Print the keys (fields) of the first document
    print("Fields in the document:")
    for key in first_doc.keys():
        print(f"- {key}: {type(first_doc[key]).__name__}")
    
    # Print a sample of the first document
    print("\nSample document:")
    print(json.dumps(first_doc, indent=2))
else:
    print("The database is empty.")

Fields in the document:
- ID: int
- name: str
- href: str
- view_count: int
- docket_number: str
- additional_docket_numbers: NoneType
- manner_of_jurisdiction: str
- first_party: str
- second_party: str
- timeline: list
- lower_court: dict
- facts_of_the_case: str
- question: str
- conclusion: str
- advocates: list
- oral_argument_audio: list
- citation: dict
- decisions: list
- first_party_label: str
- second_party_label: str
- heard_by: list
- decided_by: dict
- term: str
- location: NoneType
- opinion_announcement: list
- description: NoneType
- written_opinion: list
- related_cases: NoneType
- justia_url: str
- argument2_url: NoneType

Sample document:
{
  "ID": 53807,
  "name": "Oklahoma Tax Commission v. Citizen Band, Potawatomi Indian Tribe of Oklahoma",
  "href": "https://api.oyez.org/cases/1990/89-1322",
  "view_count": 0,
  "docket_number": "89-1322",
  "additional_docket_numbers": null,
  "manner_of_jurisdiction": "Writ of <i>certiorari</i>",
  "first_party": "Oklahoma Tax 

In [10]:
first_doc

{'ID': 53807,
 'name': 'Oklahoma Tax Commission v. Citizen Band, Potawatomi Indian Tribe of Oklahoma',
 'href': 'https://api.oyez.org/cases/1990/89-1322',
 'view_count': 0,
 'docket_number': '89-1322',
 'additional_docket_numbers': None,
 'manner_of_jurisdiction': 'Writ of <i>certiorari</i>',
 'first_party': 'Oklahoma Tax Commission',
 'second_party': 'Citizen Band, Potawatomi Indian Tribe of Oklahoma',
 'timeline': [{'event': 'Argued',
   'dates': [663228000],
   'href': 'https://api.oyez.org/case_timeline/case_timeline/36496'},
  {'event': 'Decided',
   'dates': [667548000],
   'href': 'https://api.oyez.org/case_timeline/case_timeline/36497'}],
 'lower_court': {'ID': 14,
  'name': 'United States Court of Appeals for the Tenth Circuit',
  'href': 'https://api.oyez.org/taxonomy/term/14'},
 'facts_of_the_case': '',
 'question': '',
 'conclusion': '',
 'advocates': [{'advocate': {'ID': 22617,
    'name': 'Edwin S. Kneedler',
    'href': 'https://api.oyez.org/people/edwin_s_kneedler',
   

In [13]:
import requests
from bs4 import BeautifulSoup

def fetch_and_convert_to_markdown(url):
    # Fetch the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the div with the class "primary-content"
        primary_content_div = soup.find('div', class_='primary-content')

        if primary_content_div:
            # Convert the HTML to markdown
            def html_to_markdown(soup):
                markdown = ""
                for element in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'ul', 'ol', 'li']):
                    if element.name.startswith('h'):
                        level = int(element.name[1])
                        markdown += f"{'#' * level} {element.get_text()}\n\n"
                    elif element.name == 'p':
                        markdown += f"{element.get_text()}\n\n"
                    elif element.name in ['ul', 'ol']:
                        for li in element.find_all('li'):
                            markdown += f"- {li.get_text()}\n"
                        markdown += "\n"
                return markdown

            return html_to_markdown(primary_content_div)
        else:
            return "No primary content found in the document."
    else:
        return f"Failed to fetch the URL: {response.status_code}"

# Example usage
url = "https://supreme.justia.com/cases/federal/us/498/505/"
markdown_content = fetch_and_convert_to_markdown(url)
print(markdown_content)


# 
		Okla. Tax Comm'n v. Potawatomi Tribe, 498 U.S. 505 (1991)

- Overview
- Opinions
- Materials


U.S. Supreme Court
Okla. Tax Comm'n v. Potawatomi Tribe,
498 U.S. 505 (1991)


## U.S. Supreme Court

Oklahoma Tax Commission v. Citizen
Band,

Potawatomi Indian Tribe of
Oklahoma

No. 89-1322

Argued Jan. 7, 1991

Decided Feb. 26, 1991

498 U.S. 505

Syllabus

Although, for many years, respondent Indian Tribe has sold
cigarettes at a convenience store that it owns and operates in
Oklahoma on land held in trust for it by the Federal Government, it
has never collected Oklahoma's cigarette tax on these sales. In
1987, petitioner, the Oklahoma Tax Commission (Oklahoma or
Commission), served the Tribe with an assessment letter, demanding
that it pay taxes on cigarette sales occurring between 1982 and
1986. The Tribe filed suit in the District Court to enjoin the
assessment, and Oklahoma counterclaimed to enforce the assessment
and to enjoin the Tribe from making future sales without collecti

In [15]:
from tinydb import TinyDB, Query

# Load the final.json database
final_db = TinyDB('final.json')

# Create a new database for opinions
opinions_db = TinyDB('opinions.json')

# Iterate over each document in the final.json database
for doc in final_db:
    written_opinions = doc.get('written_opinion', [])
    if written_opinions is not None:
        for opinion in written_opinions:
            justia_opinion_url = opinion.get('justia_opinion_url')
            justia_opinion_id = opinion.get('id')
            title = opinion.get('title')

            if justia_opinion_url and justia_opinion_id and title:
                # Fetch the markdown content
                markdown_content = fetch_and_convert_to_markdown(justia_opinion_url)

                # Save the fetched document in the opinions database
                opinions_db.insert({
                    'id': justia_opinion_id,
                    'title': title,
                    'content': markdown_content
                })


In [16]:
# Sample the opinions database
sampled_opinions = opinions_db.all()[:5]  # Get the first 5 documents as a sample

# Print the sampled opinions
for opinion in sampled_opinions:
    print(opinion)


{'id': 5266, 'title': 'Syllabus', 'content': '# \n\t\tOkla. Tax Comm\'n v. Potawatomi Tribe, 498 U.S. 505 (1991)\n\n- Overview\n- Opinions\n- Materials\n\n\nU.S. Supreme Court\nOkla. Tax Comm\'n v. Potawatomi Tribe,\n498 U.S. 505 (1991)\n\n\n## U.S. Supreme Court\n\nOklahoma Tax Commission v. Citizen\nBand,\n\nPotawatomi Indian Tribe of\nOklahoma\n\nNo. 89-1322\n\nArgued Jan. 7, 1991\n\nDecided Feb. 26, 1991\n\n498 U.S. 505\n\nSyllabus\n\nAlthough, for many years, respondent Indian Tribe has sold\ncigarettes at a convenience store that it owns and operates in\nOklahoma on land held in trust for it by the Federal Government, it\nhas never collected Oklahoma\'s cigarette tax on these sales. In\n1987, petitioner, the Oklahoma Tax Commission (Oklahoma or\nCommission), served the Tribe with an assessment letter, demanding\nthat it pay taxes on cigarette sales occurring between 1982 and\n1986. The Tribe filed suit in the District Court to enjoin the\nassessment, and Oklahoma counterclaimed t

In [17]:
len(opinions_db)

6939